In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import optimize

from sklearn.metrics import accuracy_score, f1_score
from pymoo.core.problem import Problem
from evoman.environment import Environment
from fitness_functions import original_fitness, individual_gain
from utils import simulation, verify_solution, init_env, run_pymoo_algorithm, initialise_script
import math


pygame 2.5.1 (SDL 2.28.2, Python 3.9.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
solutions = []
for file in os.listdir('solutions_beats_5_enemies'):
    if file.startswith('pymoo'):
        new_solution = []
        with open(f'solutions_beats_5_enemies/{file}') as f:
            solutions.append(f.read().splitlines())
            
solutions = np.array(solutions, dtype=float)

In [3]:
seed = 42
POP_SIZE = 1
ENEMIES = [1, 2, 3, 4, 5, 6, 7, 8]
n_hidden_neurons = 10


In [9]:
class objectives(Problem):
    enemies: list[int]
    env: Environment

    def __init__(self, env: Environment, n_genes: int, enemies: list[int], n_objectives):
        self.env = env
        self.enemies = enemies
        super().__init__(n_var=n_genes, n_obj=n_objectives, xl=-1, xu=1, type_var=float)

    def _evaluate(self, x: list[np.array], out, *args, **kwargs):
        #if POP_SIZE != len(x):
        #    print(f"WARNING: POP_SIZE != len(x) in evaluation step (this happens sometimes do not see why)\n"
        #          f"pop size: {POP_SIZE}; len x:{len(x)}")
        # Initialize
        dict_enemies = {}
        # Get fitness for each enemy
        for enemy in self.enemies:
            self.env.update_parameter('enemies', [enemy])

            dict_enemies[enemy] = []
            for individual_id in range(len(x)):
                dict_enemies[enemy].append(
                    simulation(self.env, x[individual_id], inverted_fitness=True, fitness_function=individual_gain))

        # Return fitness outputs for enemies
        objectives_fitness = {
            "objective_hard": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [1, 6]]) for ind_id in
                               range(len(x))],
            "objective_medium": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [2, 5, 8]]) for ind_id in
                                 range(len(x))],
            "objective_easy": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [3, 4, 7]]) for ind_id in
                               range(len(x))],
        }

        out["F"] = np.column_stack([objectives_fitness[key] for key in objectives_fitness.keys()])
        return out

In [5]:
env, n_genes = init_env('local_search_test', ENEMIES, n_hidden_neurons)
env.update_parameter('multiplemode', 'no')
problem = objectives(
        env=env,
        n_genes=n_genes,
        enemies=ENEMIES,
        n_objectives=3)

In [6]:
x_curr = solutions[[np.random.rand],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(x_curr, {})['F']
f_best, f_best_orig = np.copy(f_curr), np.copy(f_curr)

n_nbrs = 25

print(f_best_orig)

[[0.00607702 0.00324837 0.00391076]]


In [23]:
# This is the function that needs to be defined, this is how you find neighbors of solution x
# For now, it gives a neighborhood of n neighbors where every neighbor's gene has a SMALL chance to be altered SLIGHTLY.
# I could not find clear examples of algorithms for the neighbor generating functions, so this will have to do for now

def neighborhood(x, n, p_mut=0.05, sigma=0.05):
    nbrhood = []
    for _ in range(n):
        # Make new copy of solution
        new_nbr = np.copy(x)
        
        # Make probability vector for 'mutating'
        p = np.random.uniform(size=new_nbr.shape[1])
        
        # 'Mutate' genes
        new_nbr[0][p < p_mut] = new_nbr[0][p < p_mut] * np.random.normal(0, sigma)     
        
        # Add neighbor to neighborhood
        nbrhood.append(new_nbr)
    
    return nbrhood

In [32]:
done = False
solutionsChecked = 0
method = 'steepest ascent'

# Steepest ascent local search searches until no better neighbor is found
# First ascent local search searches until the first better neighbor is found
while not done:
    
    # Make new neighborhood
    Neighborhood = neighborhood(x_curr, 25, 0.05)
    
    # Evaluate every neighbor s
    for s in Neighborhood:
        solutionsChecked += 1
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except:
            # print("Infeasible solution evaluated")
            continue
        
        # If the neighbor has a better fitness evaluation, this becomes the new best
        if eval_s[0].mean() < f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
    
    # Run until no further improvements (steepest ascent)
    if np.array_equal(f_best, f_curr):
        done = True
    else:
        x_curr = np.copy(x_best)
        f_curr = np.copy(f_best)
        
        print(f"Total number of solutions checked: {solutionsChecked}")
        print(f"Best value found so far: {f_best} ({1 / f_best})")
        
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"New solution inverted fitness: {f_best} ({1 / f_best})")
print(f"Initial solution fitness: {f_best_orig} ({1 / f_best_orig})")

Total number of solutions checked: 25
Best value found so far: [[0.00694836 0.0059943  0.00650197]] ([[143.91877804 166.82505156 153.79965396]])
Final number of solutions checked: 50
New solution inverted fitness: [[0.00694836 0.0059943  0.00650197]] ([[143.91877804 166.82505156 153.79965396]])
Initial solution fitness: [[0.00607702 0.00324837 0.00391076]] ([[164.55421239 307.84687569 255.70459067]])


In [ ]:
x_curr = solutions[[0],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(solutions, {})['F']
f_best = np.copy(f_curr)

print(f_curr)

In [12]:
t = 100000
M = 5
k = 0

solutionsChecked = 0

done = False

# Same idea as steepest ascent, but if a neighbor is not better than the original solution, it still has a chance
# to become the new best solution based on some probability p.
# the rate at which this happens is controlled by the temperature T.
while not done:
    if t < 1:
        done = True
    
    m = 0
    while m < M:
        solutionsChecked += 1
        # print(f"k = {k}, m = {m}, s = {solutionsChecked} \n")
        
        N = neighborhood(x_curr, M)
        idx = np.random.randint(len(N))
        s = N[idx]        
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except Exception as e:
            break
        
        if eval_s[0].mean() <= f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
        else:
            # In literature, the 1e-7 isn't there but if you leave it out, p=1 for almost every case, so useless
            p = np.exp(-(f_curr[0].mean() - eval_s[0].mean()) / (t * 1e-7))
            test_p = np.random.uniform(0, 1)
            if test_p <= p:
                x_best = np.copy(s)
                f_best = np.copy(eval_s)
        
        m += 1
        
    k += 1
    t = 0.8 * t
    print(f"Temperature lowered to {t} (s= {solutionsChecked})")
    
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"New solution inverted fitness: {f_best} ({1 / f_best})")
print(f"Initial solution fitness: {f_best_orig} ({1 / f_best_orig})")

Temperature lowered to 80000.0 (s= 5)
Temperature lowered to 64000.0 (s= 10)
Temperature lowered to 51200.0 (s= 15)
Temperature lowered to 40960.0 (s= 20)
Temperature lowered to 32768.0 (s= 25)
Temperature lowered to 26214.4 (s= 30)
Temperature lowered to 20971.520000000004 (s= 35)
Temperature lowered to 16777.216000000004 (s= 40)
Temperature lowered to 13421.772800000004 (s= 45)
Temperature lowered to 10737.418240000005 (s= 50)
Temperature lowered to 8589.934592000003 (s= 55)
Temperature lowered to 6871.947673600003 (s= 60)
Temperature lowered to 5497.558138880003 (s= 65)
Temperature lowered to 4398.046511104002 (s= 70)
Temperature lowered to 3518.437208883202 (s= 75)
Temperature lowered to 2814.7497671065616 (s= 80)
Temperature lowered to 2251.7998136852493 (s= 85)
Temperature lowered to 1801.4398509481996 (s= 90)
Temperature lowered to 1441.1518807585599 (s= 95)
Temperature lowered to 1152.921504606848 (s= 100)
Temperature lowered to 922.3372036854785 (s= 105)
Temperature lowered to

C:\Users\beuze\AppData\Local\Temp\ipykernel_17616\3918058655.py:31: RuntimeWarning: overflow encountered in exp
  p = np.exp(-(f_curr[0].mean() - eval_s[0].mean()) / (t * 1e-7))


Temperature lowered to 8.507059173023478 (s= 210)
Temperature lowered to 6.805647338418783 (s= 215)
Temperature lowered to 5.444517870735027 (s= 220)
Temperature lowered to 4.355614296588022 (s= 225)
Temperature lowered to 3.4844914372704174 (s= 230)
Temperature lowered to 2.787593149816334 (s= 235)
Temperature lowered to 2.2300745198530674 (s= 240)
Temperature lowered to 1.784059615882454 (s= 245)
Temperature lowered to 1.4272476927059632 (s= 250)
Temperature lowered to 1.1417981541647706 (s= 255)
Temperature lowered to 0.9134385233318165 (s= 260)
Temperature lowered to 0.7307508186654532 (s= 265)
Final number of solutions checked: 265
New solution inverted fitness: [[0.00572779 0.00419921 0.00492248]] ([[174.58749516 238.14023757 203.14970662]])
Initial solution fitness: [[0.00607702 0.00324837 0.00391076]] ([[164.55421239 307.84687569 255.70459067]])
